In [84]:
import numpy as np
import pandas as pd



reading data

In [85]:
emotion_df = pd.read_csv("1-P-3-ISEAR.csv")
emotion_df.head()

,0,joy,On days when I feel close to my partner and other friends. \nWhen I feel at peace with myself and also experience a close \ncontact with people whom I regard greatly.
0,1,fear,Every time I imagine that someone I love or I ...
1,2,anger,When I had been obviously unjustly treated and...
2,3,sadness,When I think about the short time that we live...
3,4,disgust,At a gathering I found myself involuntarily si...
4,5,shame,When I realized that I was directing the feeli...


adding name to column

In [86]:
emotion_df.columns = ['sn','Target','Sentence']
emotion_df.drop('sn',inplace=True,axis =1)

In [87]:
emotion_df.head()

,Target,Sentence
0,fear,Every time I imagine that someone I love or I ...
1,anger,When I had been obviously unjustly treated and...
2,sadness,When I think about the short time that we live...
3,disgust,At a gathering I found myself involuntarily si...
4,shame,When I realized that I was directing the feeli...


In [88]:
emotion_df['Target'].value_counts()

joy        1081
sadness    1074
anger      1069
fear       1063
disgust    1059
shame      1059
guilt      1040
Name: Target, dtype: int64

the data looks fairly balance

### Text Preprocessing

converting all sentences to lower case

In [89]:
def lowercase(text):
    '''a function lowercasing all characters'''
    text = text.lower()
    return text

emotion_df['Sentence'] = emotion_df['Sentence'].apply(lowercase)

removing punctuation and number from sentences

In [90]:
import string

def remove_punct_num(text):
    '''a function for removing punctuation and number'''
    text = "".join([char for char in text if char not in string.punctuation and not char.isdigit()])
    return text


emotion_df['Sentence'] = emotion_df['Sentence'].apply(remove_punct_num)

In [91]:
emotion_df.head()

,Target,Sentence
0,fear,every time i imagine that someone i love or i ...
1,anger,when i had been obviously unjustly treated and...
2,sadness,when i think about the short time that we live...
3,disgust,at a gathering i found myself involuntarily si...
4,shame,when i realized that i was directing the feeli...


removing all stopwords

In [92]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/araul/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [93]:

def remove_stopwords(text):
    """
    Removing stopwords belonging to english language
    """
    text = [w for w in text.split() if w not in stopwords.words('english')]
    return ' '.join(text)

emotion_df['Sentence'] = emotion_df['Sentence'].apply(remove_stopwords)

In [94]:
emotion_df.head()

,Target,Sentence
0,fear,every time imagine someone love could contact ...
1,anger,obviously unjustly treated possibility elucida...
2,sadness,think short time live relate periods life thin...
3,disgust,gathering found involuntarily sitting next two...
4,shame,realized directing feelings discontent partner...


Lemmatization i.e changing words into it's root form

In [95]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.corpus import wordnet

[nltk_data] Downloading package wordnet to /home/araul/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [96]:
lemmatizer = WordNetLemmatizer()

def lemmatize(text):
    text = [lemmatizer.lemmatize(word) for word in text.split()]
    return ' '.join(text)

emotion_df['Sentence'] = emotion_df['Sentence'].apply(lemmatize)

In [97]:
emotion_df.head()

,Target,Sentence
0,fear,every time imagine someone love could contact ...
1,anger,obviously unjustly treated possibility elucida...
2,sadness,think short time live relate period life think...
3,disgust,gathering found involuntarily sitting next two...
4,shame,realized directing feeling discontent partner ...


spliting data

In [98]:
from sklearn.model_selection import train_test_split
X = emotion_df['Sentence']
y = emotion_df['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=10)


TFIDF 

It is technique to transform text into a meaningful vector of numbers.
TFIDF penalizes words that come up too often and don't really have much use. 
So it rescales the frequency of words that are common which makes scoring more balanced

In [99]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1, 2))
train_tfidf = tfidf.fit_transform(X_train)
test_tfidf = tfidf.transform(X_test)

### Model Building

In [100]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

In [101]:
lr = LogisticRegression()
lr.fit(train_tfidf,y_train)
lr.score(train_tfidf, y_train), lr.score(test_tfidf, y_test)

(0.8586299529885829, 0.5789120214909335)

In [102]:
nb = MultinomialNB()
nb.fit(train_tfidf,y_train)
nb.score(train_tfidf, y_train), nb.score(test_tfidf, y_test)

(0.8404969778374748, 0.5674949630624581)

Test prediction of random sentence

In [103]:
test_sentence = ['i am very disappointed at you']
test_sentence = tfidf.transform(test_sentence)

lr.predict(test_sentence)


array(['sadness'], dtype=object)